In [52]:
# quantum_simulation.py

import numpy as np
import qutip
from qutip_qip.circuit import QubitCircuit
from qutip_qip.device import DispersiveCavityQED

# Define qubit frequencies in GHz
frequencies = [5.0, 5.1]  # Qubit 0 and Qubit 1 frequencies

# Define qubit T1 relaxation times in microseconds
T1 = [1e-2, 1e-6]  # Qubit 0 and Qubit 1 T1 times

# Initialize QubitCircuit with 2 qubits
qc = QubitCircuit(2)
qc.add_gate("RX", 0, arg_value=np.pi)
qc.add_gate("RY", 1, arg_value=np.pi)
qc.add_gate("CNOT", targets=[1], controls=[0])

# Initialize DispersiveCavityQED processor with frequencies and T1 times
processor = DispersiveCavityQED(2, g=0.05, frequencies=frequencies, T1=T1, w0 = 9.2, nsteps = 1e5)
processor.load_circuit(qc)

# Define initial state
initial_state = qutip.basis([10, 2, 2], [0, 0, 0])

# Run the simulation with T1 relaxation
result = processor.run_state(
    initial_state,
    options=qutip.Options(nsteps=5000)
)

# Extract the final qubit state by tracing out the cavity
final_qubit_state = result.states[-1].ptrace([1, 2])

# Define the expected state after running the circuit
expected_state = qc.run(qutip.basis([2, 2], [0, 0]))

# Calculate and print fidelity
fidelity = round(qutip.fidelity(final_qubit_state, expected_state), 4)
print(fidelity)


IntegratorException: Excess work done on this call. Try to increasing the nsteps parameter in the Options class

In [24]:
import numpy as np
import qutip
from qutip_qip.circuit import QubitCircuit
from qutip_qip.device import DispersiveCavityQED

# Define the qubit circuit
qc = QubitCircuit(2)
qc.add_gate("RX", 0, arg_value=np.pi)
qc.add_gate("RY", 1, arg_value=np.pi)
qc.add_gate("CNOT", targets=[1], controls=[0])

# Qubit frequencies (in Hz or any consistent units)
qubit_frequencies = [5.0, 6.0]  # Example values

# Relaxation times (T1) in seconds
T1_times = [30e-6, 25e-6]  # Example values for qubit 0 and qubit 1

# Initialize the processor with explicit frequencies
processor = DispersiveCavityQED(2, g=0.1, omega=qubit_frequencies)

# Load the quantum circuit into the processor
processor.load_circuit(qc)

# Define collapse operators for T1 relaxation
collapse_ops = [
    np.sqrt(1 / T1_times[0]) * qutip.tensor(qutip.destroy(2), qutip.qeye(2)),  # Qubit 0 T1
    np.sqrt(1 / T1_times[1]) * qutip.tensor(qutip.qeye(2), qutip.destroy(2))   # Qubit 1 T1
]

# Initial state: ground state in the extended Hilbert space of the system
initial_state = qutip.basis([10, 2, 2], [0, 0, 0])

# Run the simulation with collapse operators to account for decoherence (T1)
result = processor.run_state(
    initial_state,
    options=qutip.Options(nsteps=5000)
)

# Trace out to get the final state of the qubits (qubits 1 and 2)
final_qubit_state = result.states[-1].ptrace([1, 2])

# Compare the fidelity of the final state with the expected output from the ideal circuit
ideal_final_state = qc.run(qutip.basis([2, 2], [0, 0]))
fidelity = qutip.fidelity(final_qubit_state, ideal_final_state)

# Print the fidelity rounded to 4 decimal places
print(round(fidelity, 4))


0.9987
